In [ ]:
# Rising Sign Precision Tracker

## Real-Time Ascendant Degree & Minute Calculator

The rising sign (Ascendant) is the fastest-moving element in astrology, changing approximately **1 degree every 4 minutes**. This notebook provides precise tracking of the Ascendant's exact degree and minute at any given time and location.

## Key Facts About Rising Sign Movement

- **Speed**: ~1° every 4 minutes
- **Complete Cycle**: 24 hours (360°)
- **Precision Required**: Degree and minute accuracy
- **Location Dependent**: Requires latitude/longitude
- **Time Sensitive**: Must account for exact time zones

## Applications

1. **Real-Time Astrology**: Track exact rising sign for current moment
2. **Precise Birth Charts**: Calculate exact Ascendant for birth times
3. **Electional Astrology**: Find optimal timing based on rising sign
4. **Research**: Study rising sign patterns and their effects
5. **API Integration**: Connect with Astrologize API for live data

## Mathematical Foundation

The Ascendant calculation involves:
- **Sidereal Time**: Based on Earth's rotation relative to stars
- **Obliquity**: Earth's axial tilt (~23.44°)
- **Local Latitude**: Observer's geographic position
- **Time Conversion**: UTC to local sidereal time


In [ ]:
// Core astronomical calculation functions for precise Ascendant tracking

// Constants for calculations
const J2000_EPOCH = 2451545.0; // Julian day for J2000.0
const DEGREES_PER_HOUR = 15; // Earth rotates 15° per hour
const MINUTES_PER_DEGREE = 4; // Rising sign changes 1° every 4 minutes
const OBLIQUITY = 23.4392911; // Mean obliquity of ecliptic for J2000

// Zodiac signs in order
const zodiacSigns = [
  'Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo',
  'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces'
];

// Default locations for testing
const locations = {
  newYork: { lat: 40.7128, lon: -74.0060, timezone: -5 },
  london: { lat: 51.5074, lon: -0.1278, timezone: 0 },
  tokyo: { lat: 35.6762, lon: 139.6503, timezone: 9 },
  losAngeles: { lat: 34.0522, lon: -118.2437, timezone: -8 },
  sydney: { lat: -33.8688, lon: 151.2093, timezone: 10 }
};

function degreesToRadians(degrees) {
  return degrees * Math.PI / 180;
}

function radiansToDegrees(radians) {
  return radians * 180 / Math.PI;
}

// Calculate Julian Day Number
function calculateJulianDay(date) {
  const year = date.getUTCFullYear();
  const month = date.getUTCMonth() + 1;
  const day = date.getUTCDate();
  const hour = date.getUTCHours();
  const minute = date.getUTCMinutes();
  const second = date.getUTCSeconds();
  
  // Convert time to decimal day
  const decimalDay = day + (hour + minute/60 + second/3600) / 24;
  
  // Julian Day calculation
  let a = Math.floor((14 - month) / 12);
  let y = year + 4800 - a;
  let m = month + 12 * a - 3;
  
  return Math.floor(365.25 * y) + Math.floor(m / 4) + decimalDay - 32083;
}

// Calculate Greenwich Mean Sidereal Time
function calculateGMST(julianDay) {
  const t = (julianDay - J2000_EPOCH) / 36525.0;
  
  // GMST in seconds
  let gmst = 24110.54841 + 8640184.812866 * t + 0.093104 * t * t - 0.0000062 * t * t * t;
  
  // Convert to hours and normalize to 0-24 range
  gmst = (gmst / 3600) % 24;
  if (gmst < 0) gmst += 24;
  
  return gmst;
}

// Calculate Local Sidereal Time
function calculateLST(gmst, longitude) {
  // Longitude in hours (divide by 15)
  const lonHours = longitude / 15;
  let lst = gmst + lonHours;
  
  // Normalize to 0-24 range
  if (lst < 0) lst += 24;
  if (lst >= 24) lst -= 24;
  
  return lst;
}

console.log("✅ Astronomical calculation functions loaded");


In [ ]:
// Precise Ascendant calculation functions

function calculateAscendant(date, latitude, longitude) {
  // Calculate Julian Day
  const jd = calculateJulianDay(date);
  
  // Calculate Greenwich Mean Sidereal Time
  const gmst = calculateGMST(jd);
  
  // Calculate Local Sidereal Time
  const lst = calculateLST(gmst, longitude);
  
  // Convert LST to degrees (15° per hour)
  const lstDegrees = lst * 15;
  
  // Calculate Ascendant using simplified formula
  // This is a basic calculation - more complex formulas account for obliquity
  const latRad = degreesToRadians(latitude);
  const oblRad = degreesToRadians(OBLIQUITY);
  
  // RAMC (Right Ascension of Midheaven) 
  const ramc = lstDegrees;
  const ramcRad = degreesToRadians(ramc);
  
  // Calculate Ascendant longitude
  const ascendantRad = Math.atan2(
    Math.cos(ramcRad),
    -Math.sin(ramcRad) * Math.cos(oblRad) - Math.tan(latRad) * Math.sin(oblRad)
  );
  
  let ascendantLongitude = radiansToDegrees(ascendantRad);
  
  // Normalize to 0-360 range
  if (ascendantLongitude < 0) ascendantLongitude += 360;
  
  return ascendantLongitude;
}

function longitudeToSignAndDegree(longitude) {
  // Normalize longitude to 0-360 range
  const normalizedLon = ((longitude % 360) + 360) % 360;
  
  // Calculate sign index (0-11)
  const signIndex = Math.floor(normalizedLon / 30);
  
  // Calculate degree within sign (0-29.999...)
  const degreeInSign = normalizedLon % 30;
  
  // Calculate degree and minute
  const degree = Math.floor(degreeInSign);
  const minute = Math.floor((degreeInSign - degree) * 60);
  const second = Math.floor(((degreeInSign - degree) * 60 - minute) * 60);
  
  return {
    sign: zodiacSigns[signIndex],
    degree: degree,
    minute: minute,
    second: second,
    exactLongitude: normalizedLon,
    degreeInSign: degreeInSign
  };
}

function calculatePreciseRisingSign(date, location) {
  const ascendantLongitude = calculateAscendant(date, location.lat, location.lon);
  const risingSignData = longitudeToSignAndDegree(ascendantLongitude);
  
  return {
    timestamp: date.toISOString(),
    location: location,
    ascendant: {
      ...risingSignData,
      timeToNextDegree: calculateTimeToNextDegree(risingSignData.degreeInSign),
      timeToNextSign: calculateTimeToNextSign(risingSignData.degreeInSign)
    }
  };
}

function calculateTimeToNextDegree(degreeInSign) {
  const remainingDegree = 1 - (degreeInSign % 1);
  const minutesRemaining = remainingDegree * MINUTES_PER_DEGREE;
  return {
    minutes: Math.floor(minutesRemaining),
    seconds: Math.floor((minutesRemaining % 1) * 60)
  };
}

function calculateTimeToNextSign(degreeInSign) {
  const remainingInSign = 30 - degreeInSign;
  const minutesRemaining = remainingInSign * MINUTES_PER_DEGREE;
  const hours = Math.floor(minutesRemaining / 60);
  const minutes = Math.floor(minutesRemaining % 60);
  return {
    hours: hours,
    minutes: minutes
  };
}

console.log("✅ Precise Ascendant calculation functions loaded");


In [ ]:
// Real-time tracking and API integration functions

function getCurrentRisingSign(locationName = 'newYork') {
  const now = new Date();
  const location = locations[locationName];
  
  if (!location) {
    console.log(`Location '${locationName}' not found. Available: ${Object.keys(locations).join(', ')}`);
    return null;
  }
  
  const risingData = calculatePreciseRisingSign(now, location);
  
  console.log(`=== Current Rising Sign (${locationName.toUpperCase()}) ===`);
  console.log(`Time: ${now.toLocaleString()}`);
  console.log(`Location: ${location.lat}°N, ${location.lon}°W`);
  console.log(`Rising Sign: ${risingData.ascendant.sign} ${risingData.ascendant.degree}°${risingData.ascendant.minute}'${risingData.ascendant.second}"`);
  console.log(`Exact Longitude: ${risingData.ascendant.exactLongitude.toFixed(6)}°`);
  console.log(`Next Degree in: ${risingData.ascendant.timeToNextDegree.minutes}m ${risingData.ascendant.timeToNextDegree.seconds}s`);
  console.log(`Next Sign in: ${risingData.ascendant.timeToNextSign.hours}h ${risingData.ascendant.timeToNextSign.minutes}m`);
  
  return risingData;
}

function trackRisingSignChanges(location = 'newYork', durationMinutes = 60, intervalMinutes = 1) {
  console.log(`=== Tracking Rising Sign Changes ===`);
  console.log(`Location: ${location}`);
  console.log(`Duration: ${durationMinutes} minutes`);
  console.log(`Interval: ${intervalMinutes} minute(s)`);
  console.log(`Expected degree changes: ~${Math.floor(durationMinutes / MINUTES_PER_DEGREE)}`);
  console.log('');
  
  const startTime = new Date();
  const trackingData = [];
  
  for (let i = 0; i <= durationMinutes; i += intervalMinutes) {
    const currentTime = new Date(startTime.getTime() + i * 60000);
    const risingData = calculatePreciseRisingSign(currentTime, locations[location]);
    
    trackingData.push({
      timeOffset: i,
      timestamp: currentTime.toISOString(),
      sign: risingData.ascendant.sign,
      degree: risingData.ascendant.degree,
      minute: risingData.ascendant.minute,
      exactDegree: risingData.ascendant.degreeInSign.toFixed(3)
    });
    
    // Print every 5 minutes or when degree changes
    if (i % 5 === 0 || (trackingData.length > 1 && 
        trackingData[trackingData.length-1].degree !== trackingData[trackingData.length-2].degree)) {
      console.log(`+${i}m: ${risingData.ascendant.sign} ${risingData.ascendant.degree}°${risingData.ascendant.minute}' (${risingData.ascendant.degreeInSign.toFixed(3)}°)`);
    }
  }
  
  return trackingData;
}

function simulateApiCall(location, timestamp) {
  // Simulate astrologize API call structure
  const risingData = calculatePreciseRisingSign(new Date(timestamp), locations[location]);
  
  return {
    success: true,
    data: {
      ascendant: {
        sign: risingData.ascendant.sign.toLowerCase(),
        degree: risingData.ascendant.degree,
        minute: risingData.ascendant.minute,
        second: risingData.ascendant.second,
        exactLongitude: risingData.ascendant.exactLongitude
      },
      calculation: {
        timestamp: timestamp,
        location: {
          latitude: locations[location].lat,
          longitude: locations[location].lon
        },
        precision: 'degree_minute_second'
      },
      nextEvents: {
        nextDegree: risingData.ascendant.timeToNextDegree,
        nextSign: risingData.ascendant.timeToNextSign
      }
    }
  };
}

// Test current rising sign
const currentRising = getCurrentRisingSign('newYork');

console.log('\n✅ Real-time tracking functions loaded');


In [ ]:
// Demonstration: Track rising sign changes over next hour

console.log("🔬 DEMONSTRATION: Rising Sign Precision Tracking");
console.log("=" .repeat(50));

// Track changes over 1 hour with 4-minute intervals
const hourlyTracking = trackRisingSignChanges('newYork', 60, 4);

console.log("\n📊 Multiple Location Comparison (Current Moment)");
console.log("=" .repeat(45));

// Compare rising signs across different locations
const locationComparison = [];
for (const locationName of Object.keys(locations)) {
  const risingData = getCurrentRisingSign(locationName);
  if (risingData) {
    locationComparison.push({
      location: locationName,
      sign: risingData.ascendant.sign,
      degree: risingData.ascendant.degree,
      minute: risingData.ascendant.minute,
      exactLongitude: risingData.ascendant.exactLongitude
    });
  }
  console.log(''); // Add spacing between locations
}

console.log("\n⏰ Precision Timing Analysis");
console.log("=" .repeat(30));

// Show how rising sign changes throughout the day
const timingAnalysis = [];
const baseTime = new Date();
baseTime.setHours(0, 0, 0, 0); // Start at midnight

console.log("Rising sign progression throughout 24 hours:");
for (let hour = 0; hour < 24; hour += 2) {
  const testTime = new Date(baseTime.getTime() + hour * 60 * 60 * 1000);
  const risingData = calculatePreciseRisingSign(testTime, locations.newYork);
  
  timingAnalysis.push({
    hour: hour,
    sign: risingData.ascendant.sign,
    degree: risingData.ascendant.degree,
    minute: risingData.ascendant.minute
  });
  
  console.log(`${hour.toString().padStart(2, '0')}:00 - ${risingData.ascendant.sign} ${risingData.ascendant.degree}°${risingData.ascendant.minute}'`);
}

console.log("\n🎯 API Integration Example");
console.log("=" .repeat(25));

// Simulate API calls for different times
const apiExamples = [
  new Date().toISOString(),
  new Date(Date.now() + 15 * 60 * 1000).toISOString(), // +15 minutes
  new Date(Date.now() + 2 * 60 * 60 * 1000).toISOString() // +2 hours
];

apiExamples.forEach((timestamp, index) => {
  const apiResponse = simulateApiCall('newYork', timestamp);
  console.log(`API Call ${index + 1} (${new Date(timestamp).toLocaleTimeString()}):`);
  console.log(`  Rising: ${apiResponse.data.ascendant.sign} ${apiResponse.data.ascendant.degree}°${apiResponse.data.ascendant.minute}'`);
  console.log(`  Next degree in: ${apiResponse.data.nextEvents.nextDegree.minutes}m ${apiResponse.data.nextEvents.nextDegree.seconds}s`);
  console.log('');
});

console.log("✅ Rising Sign Precision Tracker Complete");
console.log("Ready for real-time integration with Astrologize API");
